In [1]:
# read the dataset
import pandas as pd
import numpy as np
dataset = pd.read_csv("/Users/vambat1/Documents/Projects/Machine-Learning/05-Stock-Trend-Prediction/data/data_processing/processed_dataset.csv")

/var/folders/_3/2fm0mg0x5qdgpsw123gqb54r0000gp/T/ipykernel_58737/1847048040.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data_train = pd.DataFrame(dataset['Close'][0:int(len(dataset)*0.70)])
data_test = pd.DataFrame(dataset['Close'][int(len(dataset)*0.70):int(len(dataset))])

In [3]:
data_train.shape

(1726, 1)

In [4]:
data_test.shape

(741, 1)

In [5]:
data_train.head()

,Close
0,403.192078
1,405.351807
2,402.112152
3,400.983154
4,401.621277


In [6]:
data_test.head()

,Close
1726,324.139740
1727,320.251862
1728,314.220673
1729,320.451263
1730,316.563354


In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
scaler = MinMaxScaler(feature_range=(0,1))

In [9]:
data_train_arr = scaler.fit_transform(data_train)
data_train_arr

array([[0.63678864],
       [0.6408564 ],
       [0.63475464],
       ...,
       [0.52657543],
       [0.53173885],
       [0.51427712]])

In [10]:
x_train = []
y_train = []

In [11]:
for i in range(100, data_train_arr.shape[0]):
    x_train.append(data_train_arr[i-100:i])
    y_train.append(data_train_arr[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [12]:
# ML model

from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

In [13]:
model = Sequential()

In [14]:
model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))

In [15]:
model.add(LSTM(units=60, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

In [16]:
model.add(LSTM(units=80, activation='relu', return_sequences=True))
model.add(Dropout(0.4))

In [17]:
model.add(LSTM(units=120, activation='relu'))
model.add(Dropout(0.5))

In [18]:
model.add(Dense(units=1))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 dropout (Dropout)           (None, 100, 50)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 60)           26640     
                                                                 
 dropout_1 (Dropout)         (None, 100, 60)           0         
                                                                 
 lstm_2 (LSTM)               (None, 100, 80)           45120     
                                                                 
 dropout_2 (Dropout)         (None, 100, 80)           0         
                                                                 
 lstm_3 (LSTM)               (None, 120)               9

In [22]:
model.compile(optimizer="adam", loss='mean_squared_error')
model.fit(x_train, y_train, epochs=50)

Epoch 1/50
51/51 [==============================] - 7s 115ms/step - loss: 0.0904
Epoch 2/50
51/51 [==============================] - 6s 114ms/step - loss: 0.0151
Epoch 3/50
51/51 [==============================] - 6s 118ms/step - loss: 0.0134
Epoch 4/50
51/51 [==============================] - 6s 117ms/step - loss: 0.0115
Epoch 5/50
51/51 [==============================] - 6s 117ms/step - loss: 0.0116
Epoch 6/50
51/51 [==============================] - 6s 118ms/step - loss: 0.0109
Epoch 7/50
51/51 [==============================] - 6s 118ms/step - loss: 0.0115
Epoch 8/50
51/51 [==============================] - 6s 118ms/step - loss: 0.0094
Epoch 9/50
51/51 [==============================] - 6s 118ms/step - loss: 0.0097
Epoch 10/50
51/51 [==============================] - 6s 117ms/step - loss: 0.0087
Epoch 11/50
51/51 [==============================] - 6s 118ms/step - loss: 0.0084
Epoch 12/50
51/51 [==============================] - 6s 117ms/step - loss: 0.0082
Epoch 13/50
51/51 [======

In [23]:
model.save("keras_model.h5")

/Users/vambat1/Documents/Projects/Machine-Learning/05-Stock-Trend-Prediction/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
